***

### Logic
#### Backlog giao: Bưu cục nhận hàng giao từ tải => Giao thành công/Trả hàng
    * Không quá 120h.
#### Backlog trả: Bưu cục nhận hàng trả từ tải => Trả hàng thành công
    * Không quá 72h.
#### Backlog lấy
    * Không quá 72h?
#### Backlog luân chuyển lấy: Lấy hàng thành công => Bàn giao tải 
    * HCM -> HN: 12h.
    * Các tỉnh còn lại: 32h.
#### Backlog luân chuyển trả: Chuyển trạng thái trả hàng => Bàn giao tải
    * Giao hàng tại HCM HN: 36h.
    * Giao hàng tại các tỉnh còn lại: 56h.
***

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as df2g
import os


# change current working directory to Downloads
os.chdir('/Users/phuoc/Downloads')

# read data
export = pd.read_excel('export.xlsx')
inside = pd.read_excel('inside.xlsx')

# old_name: new_name -> replace inside headers
renamed_inside_headers = {
    'Mã đơn': 'MaDH',
    'Mã kiện': 'MaKien',
    'Kho gửi': 'KhoGui',
    'Kho nhận': 'KhoNhan',
    'Kho hiện tại': 'KhoHienTai',
    'TG đóng kiện': 'TGDongKien',
    'TG cập nhật': 'TGCapNhat',
    'TG nhận kiện': 'TGNhanKien',
    'TG kết thúc': 'TGKetThuc',
    'Trạng thái': 'TrangThaiLuanChuyen',
    'Số đơn': 'SoDon',
    'Khối lượng': 'KhoiLuong',
    'Mã niêm phong đóng': 'MaNiemPhongDong',
    'Mã niêm phong nhận': 'MaNiemPhongNhan',
    'Hình thức đóng gói': 'HinhThucDongGoi',
    'Hình thức vận chuyển': 'HinhThucVanChuyen',
    'Ghi chú': 'GhiChu'
}

# rename columns name of inside dataframe
inside = inside.rename(columns=renamed_inside_headers)

# normalize datetime of inside dataframe
# inside['TGDongKien'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

# change str to datetime
inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]] = inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S'))
inside['KhoGui'] = inside['KhoGui'].astype(int)
inside['KhoNhan'] = inside['KhoNhan'].astype(int)
inside['KhoHienTai'] = inside['KhoHienTai'].astype(int)

# normalize datetime of export dataframe
# replace all <nil> values
export = export.replace('<nil>', np.nan)

# change str to datetime
export[[
    'ThoiGianTao',
    'ThoiGianTaoChuyenDoi',
    'ThoiGianKetThucLay',
    'ThoiGianGiaoLanDau',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoHangMongMuon',
    'TGKetThucTra',
]] = export[[
        'ThoiGianTao',
        'ThoiGianTaoChuyenDoi',
        'ThoiGianKetThucLay',
        'ThoiGianGiaoLanDau',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoHangMongMuon',
        'TGKetThucTra',
]].apply(pd.to_datetime)

# set display of dataframe
pd.set_option('display.max_columns', 88)
pd.set_option('display.max_rows', 100)

# join export with inside
data = pd.merge(export, inside[['MaDH','MaKien', 'KhoGui', 'KhoNhan', 'TGNhanKien', 'TrangThaiLuanChuyen']], on='MaDH', how='left')

# codes of e-commerces exchange
shopee_codes = ['18692']
sendo_codes = ['1539', '1160902', '1160904', '1160905']
tiki_codes = ['1367']
lazada_codes = ['1041351', '9794']
ecoms = shopee_codes + sendo_codes + tiki_codes + lazada_codes

# classify types of backlog
# delivery backlog (backlog giao)
delivery_filter = data['TrangThai'].isin([
    'Đang giao hàng', 'Giao hàng không thành công'
]) | (((data['TrangThai'] == 'Lưu kho') | (data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Đang trung chuyển hàng')) &\
      (data['KhoGiao'] == data['KhoHienTai'])) | ((data['TrangThai'] == 'Chờ giao lại') & (datetime.now() - data['ThoiGianKetThucGiao'] > pd.Timedelta('24 hours')))

delivery = data.loc[delivery_filter]
delivery['LoaiBacklog'] = 'Kho giao'
internal_delivery_filter = delivery['KhoLay'] == delivery['KhoHienTai']
delivery.loc[internal_delivery_filter, 'N0'] = delivery.loc[internal_delivery_filter]['ThoiGianKetThucLay']
delivery.loc[~internal_delivery_filter, 'N0'] = delivery.loc[~internal_delivery_filter]['TGNhanKien']
delivery['N+'] = delivery['N0'] + pd.Timedelta(hours=120)
delivery['Aging'] = (datetime.today() - delivery['N+']).fillna(pd.Timedelta(hours=9999))


# return backlog (backlog trả)
returned_filter = data['TrangThai'].isin([
    'Đang hoàn hàng', 'Hoàn hàng không thành công'
    ]) | (((data['TrangThai'] == 'Chuyển hoàn') | (data['TrangThai'] == 'Đang trung chuyển hàng hoàn')) & (((~data['KhoTra'].isnull()) & (data['KhoTra'] == data['KhoHienTai'])) |\
                                                 ((data['KhoTra'].isnull()) & (data['KhoLay'] == data['KhoHienTai']))))
returned = data.loc[returned_filter]
returned['LoaiBacklog'] = 'Kho trả'
internal_return_filter = returned['KhoHienTai'] == returned['KhoGiao']
returned.loc[internal_return_filter, 'N0'] = returned.loc[internal_return_filter]['ThoiGianKetThucGiao']
returned.loc[~internal_return_filter, 'N0'] = returned.loc[~internal_return_filter]['TGNhanKien']
returned['N+'] = returned['N0'] + pd.Timedelta(hours=72)
returned['Aging'] = (datetime.today() - returned['N+']).fillna(pd.Timedelta(hours=9999))


# pickup backlog (backlog lấy)
pickup_filter = (data['TrangThai'].isin(['Chờ lấy hàng', 'Đang lấy hàng', 'Lấy không thành công'])) |\
              ((data['TrangThai'] == 'Tạo thành công') & (data['ThoiGianKetThucLay'].isnull()) &\
               (data['KhoHienTai'] == data['KhoLay']))
pickup = data.loc[pickup_filter]
pickup['LoaiBacklog'] = 'Kho lấy'
ecoms_filter = pickup['MaKH'].isin(ecoms)
pickup.loc[ecoms_filter, 'N0'] = pickup.loc[ecoms_filter]['ThoiGianTaoChuyenDoi']
pickup.loc[~ecoms_filter, 'N0'] = pickup.loc[~ecoms_filter]['ThoiGianTao']
pickup['N+'] = pickup['N0'] + pd.Timedelta(hours=72)
pickup['Aging'] = (datetime.today() - pickup['N+']).fillna(pd.Timedelta(hours=9999))

# transporting backlog (backlog luân chuyển)
trans_filter = (((data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Lưu kho')) & (data['KhoGiao'] != data['KhoHienTai'])) |\
                       ((data['TrangThai'] == 'Đang trung chuyển hàng') & (data['KhoLay'] != data['KhoGiao']))
transporting = data.loc[trans_filter]
transporting['LoaiBacklog'] = 'Kho lấy luân chuyển'
transporting_filter = (transporting['DenTinh'] == 'Hà Nội') | (transporting['DenTinh'] == 'Hồ Chí Minh')
transporting['N0'] = transporting['ThoiGianKetThucLay']
transporting.loc[transporting_filter, 'N+'] = transporting.loc[transporting_filter]['N0']+ pd.Timedelta(hours=12)
transporting.loc[~transporting_filter, 'N+'] = transporting.loc[~transporting_filter]['N0'] + pd.Timedelta(hours=32)
transporting['Aging'] = (datetime.today() - transporting['N+']).fillna(pd.Timedelta(hours=9999))


# returned_transporting backlog (backlog luân chuyển trả hàng)
rt_filter = ((data['TrangThai'].isin(['Chuyển hoàn', 'Chờ chuyển hoàn']) | (data['TrangThai'] == 'Đang trung chuyển hàng hoàn')) &\
             ((data['KhoTra'].isnull() & (data['KhoLay'] != data['KhoHienTai'])) |
              (~data['KhoTra'].isnull() & (data['KhoTra'] != data['KhoHienTai']))))
r_transporting = data.loc[rt_filter]
r_transporting['LoaiBacklog'] = 'Kho giao luân chuyển'
r_transporting_filter = (r_transporting['DenTinh'] == 'Hà Nội') | (r_transporting['DenTinh'] == 'Hồ Chí Minh')
r_transporting['N0'] = r_transporting['ThoiGianKetThucGiao']
r_transporting.loc[r_transporting_filter, 'N+'] = r_transporting.loc[r_transporting_filter]['N0'] + pd.Timedelta(hours=36)
r_transporting.loc[~r_transporting_filter, 'N+'] = r_transporting.loc[~r_transporting_filter]['N0'] + pd.Timedelta(hours=56)
r_transporting['Aging'] = (datetime.today() - r_transporting['N+']).fillna(pd.Timedelta(hours=9999))


# composite data
data = delivery.append([returned, pickup, transporting, r_transporting])
# set e-commerces column to NaN
data['ECommerces'] = np.nan


# name of e-commerces exchange
# shopee
data.loc[data['MaKH'].isin(shopee_codes), 'ECommerces'] = 'Shopee'
# sendo
data.loc[data['MaKH'].isin(sendo_codes), 'ECommerces'] = 'Sendo'
# tiki
data.loc[data['MaKH'].isin(tiki_codes), 'ECommerces'] = 'Tiki'
# lazada
data.loc[data['MaKH'].isin(lazada_codes), 'ECommerces'] = 'Lazada'

# ToanTrinh
data['Aging_ToanTrinh'] = np.nan
ecoms_filter = data['MaKH'].isin(ecoms)
data.loc[ecoms_filter, 'Aging_ToanTrinh'] = datetime.today() - data[ecoms_filter]['ThoiGianTaoChuyenDoi']
data.loc[~ecoms_filter, 'Aging_ToanTrinh'] = datetime.today() - data[~ecoms_filter]['ThoiGianTao']
# reset the index of datafram
data.reset_index(inplace=True)
del data['index']
data['KhoGui'].astype('int64', errors='ignore')
data['NgayHienTai'] = datetime.today().strftime('%Y-%m-%d')
data
# data['KhoGui']

,MaDH,MaDHShop,ThoiGianTao,ThoiGianTaoChuyenDoi,ThoiGianKetThucLay,ThoiGianGiaoLanDau,ThoiGianKetThucGiao,ThoiGianGiaoHangMongMuon,TGKetThucTra,ThongTinTraHang,TuQuan,TuTinh,TuVung,DenQuan,DenTinh,DenVung,KhoLay,KhoGiao,KhoHienTai,KhoTra,TrangThai,TrongLuong,KichThuoc,MaKH,GhiChuGHN,GhiChu,GoiCuoc,SoLanLay,SoLanGiao,SoLanTra,TinhTrangLuanChuyen,MaNVLay,MaNVGiao,TenNguoiGui,MaDiaChiLay,MaKien,KhoGui,KhoNhan,TGNhanKien,TrangThaiLuanChuyen,LoaiBacklog,N0,N+,Aging,ECommerces,Aging_ToanTrinh,NgayHienTai
0,FHPAFQS4,KV1293243226366,2020-08-01 10:57:01,2020-08-01 10:57:00,2020-08-01 16:10:36,2020-08-04 09:17:38,NaT,2020-08-04 23:00:00,NaT,NaN,Quận Hoàng Mai,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 25 Trương Định-Q.Hai Bà Trưng-HN 02,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,NaN,Đang giao hàng,500,1x1x1,518873,NaN,NaN,2 Ngày,1,1,0,0,1714228,9999,'ptlink',2267105,20NRU94012,2329.0,2275.0,2020-08-03 14:07:25,Kết thúc,Kho giao,2020-08-03 14:07:25,2020-08-08 14:07:25,29 days 10:53:00.955962,NaN,36 days 14:03:25.339699,2020-09-07
1,8FHSAHAY5K,KV1293213386351,2020-08-13 20:36:59,2020-08-14 08:00:00,2020-08-14 19:07:15,2020-08-17 07:55:13,NaT,2020-08-19 01:06:00,NaT,NaN,Quận Hoàn Kiếm,Hà Nội,G,Quận 4,Hồ Chí Minh,E,Bưu Cục 55C Nguyễn Khoái-Q.Hai Bà Trưng-HN,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 55C Nguyễn Khoái-Q.Hai Bà Trưng-HN,Giao hàng không thành công,500,1x1x1,518873,'17/08/2020 06:39:33 - GHN-DFC1A1|Khách đổi đị...,'',2 Ngày,1,1,0,NaN,1883870,NaN,'maiphuongbuiboutique',1228196,20WMZ58349,2329.0,2275.0,2020-08-16 14:17:02,Kết thúc,Kho giao,2020-08-16 14:17:02,2020-08-21 14:17:02,16 days 10:43:23.955962,NaN,24 days 04:23:27.339699,2020-09-07
2,8FHX4Q19R7,2416068723798016203,2020-08-14 14:33:00,2020-08-14 14:33:00,2020-08-15 11:49:31,NaT,2020-09-03 19:24:23,2020-08-18 08:44:00,NaT,'Hàng giao chậm quá thời gian cam kết',Quận Tân Phú,Hồ Chí Minh,E,Quận 7,Hồ Chí Minh,E,Bưu Cục 20/82-84 Bờ Bao Tân Thắng-Q.Tân Phú-HCM,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 20/82-84 Bờ Bao Tân Thắng-Q.Tân Phú-HCM,Chờ giao lại,2000,10x10x10,18692,'14/08/2020 08:19:42 - GHN-PFA1A0|THAY ĐỔI THÔ...,'Không cho xem hàng. Khối lượng nhận hàng tối ...,1 Ngày,2,0,0,NaN,253168,NaN,'Cửa hàng Trung Nghĩa',1154025,20MPV10687,2329.0,2275.0,2020-08-16 01:25:19,Kết thúc,Kho giao,2020-08-16 01:25:19,2020-08-21 01:25:19,16 days 23:35:06.955962,Shopee,23 days 10:27:26.286466,2020-09-07
3,8FH44NYQX9,NaN,2020-08-16 11:11:00,2020-08-16 11:11:00,2020-08-27 18:02:52,2020-08-31 08:39:21,NaT,2020-09-03 07:20:00,NaT,NaN,Huyện Đình Lập,Lạng Sơn,A,Quận 7,Hồ Chí Minh,E,Bưu Cục 35 Lục Giàng-Cao Lộc-Lạng Sơn,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 35 Lục Giàng-Cao Lộc-Lạng Sơn,Lưu kho,500,1x1x1,518873,'24/08/2020 05:08:28 - GHN-PCB0B1|Người gửi yê...,'',3 Ngày,1,3,0,NaN,1874156,NaN,'huyenminshopls',1242420,20MNZ17873,2329.0,2275.0,2020-08-29 20:32:20,Kết thúc,Kho giao,2020-08-29 20:32:20,2020-09-03 20:32:20,3 days 04:28:05.955962,NaN,21 days 13:49:26.339699,2020-09-07
4,8FH7DKUDAU,5f408fdde0314d0001f935a4_59,2020-08-22 10:26:08,2020-08-22 10:26:09,2020-08-27 10:28:45,2020-08-29 15:12:35,NaT,2020-08-31 23:00:00,NaT,NaN,Huyện Yên Mỹ,Hưng Yên,B,Quận 7,Hồ Chí Minh,E,Bưu Cục Yên Mỹ-Hưng Yên,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục Yên Mỹ-Hưng Yên,Lưu kho,1000,10x80x10,2522198,'29/08/2020 09:16:33 - GHN-DFC1A0|THAY ĐỔI THÔ...,'XEM HÀNG ĐÚNG KO NHẬN TT 30K PHÍ SHIP NGA ',3 Ngày,1,2,0,NaN,1879544,NaN,'Tổng kho gia dụng ',1288591,20DAJ52201,2268.0,2275.0,2020-08-29 14:58:39,Kết thúc,Kho giao,2020-08-29 14:58:39,2020-09-03 14:58:39,3 days 10:01:46.955962,NaN,15 days 14:34:18.339699,2020-09-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6122,8FHHFRKSQ5,2428219052339201996,2020-08-31 08:53:32,2020-08-31 0

In [3]:
# export data backlog to google sheets
# data export
data_compact = data[[
    'NgayHienTai',
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanLay',
    'SoLanGiao',
    'SoLanTra',
    'LoaiBacklog',
    'N0',
    'N+',
    'Aging',
    'Aging_ToanTrinh',
    'ECommerces',
    'TrangThaiLuanChuyen',
]]

# define scope of google api json keyfile
SCOPES = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# credentials to google service account
credentials = ServiceAccountCredentials.from_json_keyfile_name('ggapi_key.json', scopes=SCOPES)
# authorize with gspread
gc = gspread.authorize(credentials)

SPREADSHEET_ID = '1_UlAs1Ed6rxl1TNM8Vv2fnEsRT61BOWTVzCMtd1hwME'  # id of sheets (on url)
WKS_NAME = 'Backlog_bot'
active_sheets = gc.open_by_key(SPREADSHEET_ID)
wks = active_sheets.worksheet(WKS_NAME)
data_upload = data_compact.loc[data_compact['Aging'] >= pd.Timedelta(hours=0)]
# update data to google sheets
wks.clear()
df2g.upload(
    data_upload,
    SPREADSHEET_ID,
    WKS_NAME,
    credentials=credentials,
    row_names=False,
)


<Worksheet 'Backlog_bot' id:181055436>

In [4]:
# read dataframe from google sheet
from df2gspread import gspread2df as g2df

df_gspread = g2df.download(gfile=SPREADSHEET_ID, wks_name=WKS_NAME, credentials=credentials, col_names=True, row_names=False)

df_gspread

,NgayHienTai,MaDH,KhoHienTai,TrangThai,GhiChuGHN,SoLanLay,SoLanGiao,SoLanTra,LoaiBacklog,N0,N+,Aging,Aging_ToanTrinh,ECommerces,TrangThaiLuanChuyen
0,2020-09-07,FHPAFQS4,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Đang giao hàng,nan,1,1,0,Kho giao,2020-08-03 14:07:25,2020-08-08 14:07:25,29 days 10:53:00.955962,36 days 14:03:25.339699,nan,Kết thúc
1,2020-09-07,8FHSAHAY5K,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Giao hàng không thành công,'17/08/2020 06:39:33 - GHN-DFC1A1|Khách đổi đị...,1,1,0,Kho giao,2020-08-16 14:17:02,2020-08-21 14:17:02,16 days 10:43:23.955962,24 days 04:23:27.339699,nan,Kết thúc
2,2020-09-07,8FHX4Q19R7,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Chờ giao lại,'14/08/2020 08:19:42 - GHN-PFA1A0|THAY ĐỔI THÔ...,2,0,0,Kho giao,2020-08-16 01:25:19,2020-08-21 01:25:19,16 days 23:35:06.955962,23 days 10:27:26.286466,Shopee,Kết thúc
3,2020-09-07,8FH44NYQX9,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Lưu kho,'24/08/2020 05:08:28 - GHN-PCB0B1|Người gửi yê...,1,3,0,Kho giao,2020-08-29 20:32:20,2020-09-03 20:32:20,3 days 04:28:05.955962,21 days 13:49:26.339699,nan,Kết thúc
4,2020-09-07,8FH7DKUDAU,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Lưu kho,'29/08/2020 09:16:33 - GHN-DFC1A0|THAY ĐỔI THÔ...,1,2,0,Kho giao,2020-08-29 14:58:39,2020-09-03 14:58:39,3 days 10:01:46.955962,15 days 14:34:18.339699,nan,Kết thúc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4185,2020-09-07,8FHHFRKSQ5,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Đang trung chuyển hàng hoàn,'31/08/2020 10:23:01 - GHN-PFA1A0|THAY ĐỔI THÔ...,2,1,0,Kho giao luân chuyển,2020-09-01 19:59:24,2020-09-03 07:59:24,3 days 17:01:02.215324,6 days 16:06:54.286466,Shopee,Đã nhận
4186,2020-09-07,8FHLYQ74R3,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Chuyển hoàn,'22/08/2020 08:26:38 - GHN-DFC1A0|THAY ĐỔI THÔ...,1,4,0,Kho giao luân chuyển,2020-09-01 16:22:42,2020-09-03 04:22:42,3 days 20:37:44.215324,18 days 15:47:12.339699,nan,Đã nhận
4187,2020-09-07,GHNMP0000716014VNA,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Chuyển hoàn,'03/09/2020 13:37:33 - GHN-DFC1A4|KHÔNG LIÊN L...,1,2,0,Kho giao luân chuyển,2020-09-03 20:28:05,2020-09-05 08:28:05,1 days 16:32:21.215324,5 days 13:34:00.286466,Lazada,Kết thúc
4188,2020-09-07,GHNMP0000711703VNA,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Chuyển hoàn,'03/09/2020 19:24:08 - GHN-DFC1A7|THAY ĐỔI THÔ...,1,1,0,Kho giao luân chuyển,2020-09-03 20:28:02,2020-09-05 08:28:02,1 days 16:32:24.215324,5 days 16:25:12.286466,Lazada,Khởi tạo


In [5]:
# Inventory report
# from df2gspread import df2gspread as df2g
data_inventory = data.loc[(data['LoaiBacklog'] == 'Kho giao') & (data['TrangThai'] != 'Đang giao hàng')]
data_inventory['LoaiXuLy'] = 'Chưa giao lại'
chuagiao_filter = data_inventory['SoLanGiao'].isnull()
giaoloi_filter = ~data_inventory['GhiChuGHN'].isnull() & data_inventory['GhiChuGHN'].str.contains(datetime.today().strftime('%d/%m/%Y'))
data_inventory.loc[chuagiao_filter, 'LoaiXuLy'] = 'Chưa giao lần nào'
data_inventory.loc[giaoloi_filter, 'LoaiXuLy'] = 'Giao lỗi'

def convert_to_day(x):
    try:
        return x.strftime('%Y-%m-%d')
    except ValueError:
        return np.nan

data_inventory['N_ve_kho'] = data_inventory['N0'].apply(lambda x: convert_to_day(x))

del data_inventory['N0']
del data_inventory['N+']
del data_inventory['Aging']
del data_inventory['NgayHienTai']
del data_inventory['GhiChu']
# data_inventory
# to csv file
data_inventory.to_csv('rp_giao.csv', index=False, encoding='utf-8-sig')
# data_inventory.to_excel('report_giao.xlsx')

# pivot data and push to google sheet
table1 = pd.pivot_table(data_inventory, values='MaDH', index=['LoaiXuLy'], aggfunc='count')
table2 = pd.pivot_table(data_inventory, values='MaDH', index=['LoaiXuLy'], columns=['N_ve_kho'], aggfunc='count', fill_value=0)

# WKS_INVENTORY = 'RPGiao_bot'
# inventory_wks = gc.open_by_key(SPREADSHEET_ID).worksheet(WKS_INVENTORY)
# # inventory_wks.clear()  # clear old data
# df2g.upload(
#     table1,
#     SPREADSHEET_ID,
#     WKS_INVENTORY,
#     credentials=credentials,
#     row_names=True,
#     col_names=True
# )

# df2g.upload(
#     table2,
#     SPREADSHEET_ID,
#     WKS_INVENTORY,
#     credentials=credentials,
#     row_names=True,
#     col_names=True,
#     start_cell='A10',
# )

In [6]:
# from datetime import datetime

# dt = datetime.strptime('08/08/2020 14:10:12', '%d/%m/%Y %H:%M:%S')
# print(dt)
# def iter_df(df):
#     """Generator for df."""
#     for val in df.columns:
#         yield val
#     for row in df.to_numpy():
#         for val in row:
#             if pd.isna(val):
#                 yield ''
#             else:
#                 yield val

In [7]:
# to excel
# data_to_excel = data[[
#     'MaDH',
#     'KhoHienTai',
#     'TrangThai',
#     'GhiChuGHN',
#     'SoLanLay',
#     'SoLanGiao',
#     'SoLanTra',
#     'LoaiBacklog',
#     'N0',
#     'N+',
#     'Aging',
#     'Aging (hours)',
#     'e-commerces',
# ]]

# data_to_excel.to_excel('backlog.excel', sheet='Sheet1')